<a href="https://colab.research.google.com/github/parksj0923/KORartilleryman/blob/master/Artillery/osam(military)/osam_machine_learning/house_price_predict/Final_evaluation(House_Price_Predict).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

df_train = pd.read_csv("train.csv")

df_missing = df_train.isnull().sum()
df_missing = df_missing.sort_values(ascending=False)

df_missing = df_missing[df_missing >0]
# missing data가 있는 feature를 제거
df_train = df_train.drop(df_missing.index.tolist(),axis=1)

numeric = [df_train.dtypes[f] for f in df_train.columns]
numeric = [f for f in df_train.columns if df_train.dtypes[f] !='object']
numeric.remove('Id')
numeric.remove('SalePrice')

categorical = [f for f in df_train.columns if df_train.dtypes[f] =='object']

df_order = pd.DataFrame()
feature_name = "ExterQual"
df_order['val'] = df_train[feature_name].unique()
df_order.index = df_order.val

df_order['mean_price'] = df_train[[feature_name, 'SalePrice']].groupby(feature_name).mean()['SalePrice']
df_order = df_order.sort_values('mean_price')
df_order['rank'] = range(1,len(df_order)+1)
df_order = df_order['rank'].to_dict()
#원본 데이터는 나두는게 좋으니 새로운 feature를 만들어서 추가
#apply는 원하는 데이터의 한 row식 접근하면서 새로운 colum에 추가할 수 있음
df_train[feature_name + '_rank'] = df_train.apply(lambda x: df_order[x[feature_name]],axis=1)

In [ ]:
def add_rank_feature (feature_name, rank_feature_name):
  df_order = pd.DataFrame()
  df_order['val'] = df_train[feature_name].unique()
  df_order.index = df_order.val

  df_order['mean_price'] = df_train[[feature_name, 'SalePrice']].groupby(feature_name).mean()['SalePrice']
  df_order['rank'] = range(1,len(df_order)+1)
  df_order = df_order['rank'].to_dict()

  df_train[feature_name + '_rank'] = df_train.apply(lambda x: df_order[x[feature_name]],axis=1)

In [ ]:
rank_features = []
for feature_name in categorical:
  
  rank_feature_name = feature_name + '_rank'
  print(feature_name, '-->', rank_feature_name)
  add_rank_feature(feature_name, rank_feature_name)

  rank_features.append(rank_feature_name)

MSZoning --> MSZoning_rank
Street --> Street_rank
LotShape --> LotShape_rank
LandContour --> LandContour_rank
Utilities --> Utilities_rank
LotConfig --> LotConfig_rank
LandSlope --> LandSlope_rank
Neighborhood --> Neighborhood_rank
Condition1 --> Condition1_rank
Condition2 --> Condition2_rank
BldgType --> BldgType_rank
HouseStyle --> HouseStyle_rank
RoofStyle --> RoofStyle_rank
RoofMatl --> RoofMatl_rank
Exterior1st --> Exterior1st_rank
Exterior2nd --> Exterior2nd_rank
ExterQual --> ExterQual_rank
ExterCond --> ExterCond_rank
Foundation --> Foundation_rank
Heating --> Heating_rank
HeatingQC --> HeatingQC_rank
CentralAir --> CentralAir_rank
KitchenQual --> KitchenQual_rank
Functional --> Functional_rank
PavedDrive --> PavedDrive_rank
SaleType --> SaleType_rank
SaleCondition --> SaleCondition_rank


In [ ]:
df_processed = df_train[rank_features + numeric]
df_processed['SalePrice'] = df_train['SalePrice']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
corrmat = df_processed.corr()
df_features = corrmat.sort_values(by=['SalePrice'])['SalePrice']
NUM_FEATURES = 30
x = df_processed[df_features[-NUM_FEATURES-1:-1].index.tolist()]
y = df_processed['SalePrice']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1, random_state=42)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
#k개 각각의 데이터를 전처리하기 귀찮으니 전처리를 먼저 하고 간다
# min_max scaler는 각각의 칼럼의 범위가 제각각이니까 0-1사이로 맞춰주고 한다
#data preprocessing
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(x)
scaled_x = x_min_max_scaler.transform(x)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(np.array(y).reshape(-1,1))
scaled_y = y_min_max_scaler.transform(np.array(y).reshape(-1,1))

In [ ]:
# save numpy data
with open('x.npy','wb') as f:
  np.save(f,x)

with open('y.npy','wb') as f:
  np.save(f,y)

with open('scaled_x.npy','wb') as f:
  np.save(f,scaled_x)

with open('scaled_y.npy','wb') as f:
  np.save(f,scaled_y)  

In [ ]:
from sklearn.model_selection import KFold
# K-fold cross validation
K = 10
kf = KFold(n_splits=K)

##Linear regression (Lasso) 
y = Xw 선형모델


In [ ]:
from sklearn.linear_model import Lasso
from sklearn import metrics
rmses = []
for train_index, test_index in kf.split(scaled_x):
  #print("Test:", test_index, "trains:", train_index)

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  #print(x_train,shape, x_test.shape)
  #print(y_train.shape, y_test.shape)

  #training
  model = Lasso()
  model = model.fit(scaled_x_train, scaled_y_train)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))  #0-1
  pred = y_min_max_scaler.inverse_transform(pred)  #원상복구된 스케일
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

68479.61848137801
80011.92820033105
79275.65958882873
82868.15912800275
96386.20490927555
76808.56629102639
72198.44861652121
71213.04254017191
90347.61467595294
72613.27897372228
average rmse: 79020.25214052107


##MLP(Multi-layer Perceptron)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

rmses = []
for train_index, test_index in kf.split(scaled_x):

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]


  #training
  model = keras.Sequential(
     [
      keras.Input(shape=scaled_x_train.shape[-1]),
      layers.Dense(96, activation='relu'),
      layers.Dense(48, activation='relu'),
      layers.Dense(1)
     ]
  )

  model.compile(loss = 'mse', optimizer = 'adam')
  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)
  model.fit(scaled_x_train, scaled_y_train, batch_size=4, epochs=150,
            callbacks=[early_stopping_callback], validation_split=0.05)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))  #0-1
  pred = y_min_max_scaler.inverse_transform(pred)  #원상복구된 스케일
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

Epoch 1/150
312/312 [==============================] - 1s 2ms/step - loss: 0.0055 - val_loss: 0.0040
Epoch 2/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0035 - val_loss: 0.0020
Epoch 3/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 4/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0027 - val_loss: 0.0020
Epoch 5/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0037
Epoch 6/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0022 - val_loss: 0.0022
Epoch 7/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0022
Epoch 8/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 9/150
312/312 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0032
385437388.9109486
Epoch 1/150
312/312 [==============================] - 1s 2ms/step - loss

##Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeRegressor
rmses = []
for train_index, test_index in kf.split(scaled_x):
  #print("Test:", test_index, "trains:", train_index)

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  #print(x_train,shape, x_test.shape)
  #print(y_train.shape, y_test.shape)

  #training
  model = DecisionTreeRegressor(random_state = 0)
  model = model.fit(scaled_x_train, scaled_y_train)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

34070.24666059374
36792.7899139313
38478.76115412554
48437.07536071929
43629.418260220555
34903.48882540429
32715.948804354426
38051.52842228641
59230.083416528454
38853.31003934798
average rmse: 40516.2650857512


##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rmses = []
for train_index, test_index in kf.split(scaled_x):
  #print("Test:", test_index, "trains:", train_index)

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  #print(x_train,shape, x_test.shape)
  #print(y_train.shape, y_test.shape)

  #training
  model = RandomForestRegressor(random_state = 0, n_estimators = 200)
  model = model.fit(scaled_x_train, scaled_y_train)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


26474.567325638072


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


27462.385207784337


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


...

24177.79106533606


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


40942.91436992498


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  from ipykernel import kernelapp as app


27919.82190051351
average rmse: 29493.183294406957


##AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
rmses = []
for train_index, test_index in kf.split(scaled_x):
  #print("Test:", test_index, "trains:", train_index)

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  #print(x_train,shape, x_test.shape)
  #print(y_train.shape, y_test.shape)

  #training
  model = AdaBoostRegressor(random_state = 0, n_estimators = 200)
  model = model.fit(scaled_x_train, scaled_y_train)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


30670.99212382827


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


31297.310045005834


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

30623.153644286078


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


46314.34158762049


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


34231.72422634499
average rmse: 35199.08013022338


##XGBoost

In [ ]:
from xgboost import XGBRegressor
rmses = []
for train_index, test_index in kf.split(scaled_x):
  #print("Test:", test_index, "trains:", train_index)

  scaled_x_train, scaled_x_test = scaled_x[train_index], scaled_x[test_index]
  scaled_y_train, scaled_y_test = scaled_y[train_index], scaled_y[test_index]
  y_test = y[test_index]

  #print(x_train,shape, x_test.shape)
  #print(y_train.shape, y_test.shape)

  #training
  model = XGBRegressor(random_state = 0, n_estimators = 200)
  model = model.fit(scaled_x_train, scaled_y_train)

  #evaluation
  pred = model.predict(scaled_x_test).reshape((-1,1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test,pred))

  print(rmse)
  print("===============================")
  rmses.append(rmse)
print("average rmse:", np.mean(rmses))

[11:14:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
22570.376747612765
[11:14:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
24646.83596423113
[11:14:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
23302.28305688471
[11:14:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
43276.780661287245
[11:14:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
30420.16603047583
[11:14:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
25190.01461840584
[11:14:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
2